In [1]:
import pandas as pd 
from sklearn.pipeline import Pipeline
from datacleaning.FetchData import FetchData
from datacleaning.CleanData import CleanData
from datetime import datetime
import plotly.express as px

pd.set_option("display.max_columns", 100)

In [2]:
FetchData.scan_save_all_records()
CleanData.clean_save_raw_data()

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd 
import numpy as np 


class SortDropCast(BaseEstimator, TransformerMixin):
    """
    This pipeline step will sort values by field "connectTime",
    drop columns "user_email", "slrpPaymentId", 
    and cast columns "cumEnergy_Wh", "peakPower_W" as float values. 
    """
    def fit(self, X, y=None):
        return self

    @staticmethod
    def transform(X) -> pd.DataFrame:
        X = X.sort_values(by="connectTime").drop(columns=["user_email", "slrpPaymentId"]).reset_index(drop=True)
        X["cumEnergy_Wh"] = X["cumEnergy_Wh"].astype(float)
        X["peakPower_W"] = X["peakPower_W"].astype(float)
        return X


class HelperFeatureCreation(BaseEstimator, TransformerMixin):
    """
    This pipeline step will drop any records that contain 0 for 
    "peakPower_W" or "cumEnergy_Wh". Two additional columns will be created:
    "reqChargeTime" and "finishChargeTime".
    """
    def fit(self, X, y=None):
        return self

    @staticmethod
    def transform(X) -> pd.DataFrame:
        X = X.loc[(X["peakPower_W"] != 0) & (X["cumEnergy_Wh"] != 0)]
        X = X.assign(reqChargeTime_h=(X["cumEnergy_Wh"] / X["peakPower_W"]))
        X = X.assign(connectTime=(pd.to_datetime(X["connectTime"])))
        X = X.assign(
            finishChargeTime=(X["connectTime"] + pd.to_timedelta(X['reqChargeTime_h'], unit='hours').round("s"))
        )
        return X 


class CreateNestedSessionTimeSeries(BaseEstimator, TransformerMixin):
    """
    This pipeline step will create a time series for each session. Two new columns will be created, 
    "time_vals" and "power_vals", respective lists for a time and power demand. "time_vals" are rounded to the 
    closest 5 min. 
    """ 
    def __init__(self) -> None:
        super().__init__()
    
    def fit(self, X, y=None):
        return self 

    def transform(self, X) -> pd.DataFrame:
        self.ts_df = pd.DataFrame(columns=["time_vals", "power_vals"])
        X.apply(self.__create_ts, axis=1)
        X = pd.concat([X.reset_index(), self.ts_df], axis=1)
        return X

    def __create_ts(self, session):

        date_range = pd.date_range(start=session["connectTime"], end=session["finishChargeTime"], freq="5min").to_list()
        power_vals = np.ones(len(date_range)) * session["peakPower_W"]
        
        temp_df = pd.DataFrame([[date_range, power_vals]], columns=self.ts_df.columns)
        self.ts_df = pd.concat([self.ts_df, temp_df], ignore_index=True)


class FeatureCreation(BaseEstimator, TransformerMixin):
    """
    This pipeline step will create an "energy_demand_kWh" and "peak_power_W" column. 
    The name of the dataframe's index will be set to "time", and "day" and "month" columns 
    will be created. 
    """
    def fit(self, X, y=None):
        return self 

    @ staticmethod
    def transform(X) -> pd.DataFrame:
        X["energy_demand_kWh"] = (X["avg_power_demand_W"]/1000)/12
        # for the highest granularity, peak power is equal to the power demand
        # (different for different granularities though)
        X["peak_power_W"] = X["avg_power_demand_W"] 
        X.index.name = "time"
        X["day"] = X.index.day_name()
        X["month"] = X.index.month_name()
        return X


In [11]:
raw = pd.read_csv("data/raw_data.csv")
raw = raw.sort_values("connectTime")
raw["connectTime"] = pd.to_datetime(raw["connectTime"])
raw

,Unnamed: 0,vehicle_maxChgRate_W,peakPower_W,sch_centsPerHr,connectTime,user_email,vehicle_model,Duration,userId,regular,Deadline,startChargeTime,sch_centsPerOverstayHr,sch_centsPerKwh,choice,siteId,estCost,slrpPaymentId,DurationHrs,dcosId,lastUpdate,energyReq_Wh,power,stationId,defaultDeadline,scheduled,cumEnergy_Wh,reg_centsPerHr
2629,2629,6600,6335,9.0,2020-11-05 10:30:16,yossarianassyrian@gmail.com,500e,0 days 03:43:57,605,1,NaN,2020-11-05T10:31:09,200.0,15.0,REGULAR,23,5.35224,6b108a9e64224989135baca2cffa2c596359c45ef2c0f0...,3.73249,24,2020-11-05T14:15:06,NaN,"[{'power_W': Decimal('6259'), 'timestamp': Dec...",7,1969-12-31T16:00:00,0,3281,130.0
1376,1376,24000,7005,3.0,2020-11-11 07:39:55,sohum@ucsd.edu,Model 3,0 days 06:50:07,486,1,NaN,2020-11-11T07:39:59,200.0,15.0,REGULAR,23,10.75291,7e40611ff8f81771592628312547eedede36157d5aee63...,6.83527,26,2020-11-11T14:30:06,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",3,2020-11-12T03:11:00,0,33458,150.0
2659,2659,3600,3450,3.0,2020-11-13 16:19:55,kylebhaas@berkeley.edu,Volt,0 days 20:40:02,620,0,2020-11-14T04:15:00,2020-11-13T16:20:06,300.0,12.0,SCHEDULED,25,29.32211,5475d7bfb9fdca49e239f14c88bc0ac50a8a26976a3e44...,20.66722,30,2020-11-14T13:00:08,18400.0,"[{'power_W': Decimal('0'), 'timestamp': Decima...",12,2020-11-14T04:11:00,1,15216,180.0
675,675,7200,6889,3.0,2020-11-14 23:47:06,rayconstantino.me@gmail.com,Bolt,0 days 02:12:51,618,1,NaN,2020-11-14T23:47:16,400.0,18.0,REGULAR,23,3.82125,add3a0ba4ca2d77670c3d08bbfb1c0903774c48d08d305...,2.21416,31,2020-11-15T02:00:07,NaN,"[{'power_W': Decimal('6889'), 'timestamp': Dec...",6,1969-12-31T16:00:00,0,14378,150.0
353,353,6000,6852,NaN,2020-11-16 11:38:44,khuffman@health.ucsd.edu,B-Class Electric Drive,0 days 03:12:45,623,1,NaN,2020-11-16T11:42:22,NaN,NaN,REGULAR,23,NaN,NaN,3.21249,32,2020-11-16T14:55:07,NaN,"[{'power_W': Decimal('6813'), 'timestamp': Dec...",9,2020-11-17T04:11:00,0,12484,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2101,7000,3818,127.0,2023-02-01 07:43:25,grosales@berkeley.edu,Leaf,0 days 05:16:30,1067,1,NaN,2023-02-01T07:43:40,400.0,0.0,REGULAR,25,10.20599,277332d81349a2654cad7dbeb0f563b61fecbfaa794fcb...,5.27500,4101,2023-02-01T13:00:10,NaN,"[{'power_W': Decimal('3813'), 'timestamp': Dec...",11,2023-02-01T13:30:00,0,12932,184.0
1565,1565,7000,6855,250.0,2023-02-01 09:38:09,stephanie.tomasco@alumni.berkeley.edu,Model Y,0 days 02:20:10,880,1,NaN,2023-02-01T09:40:05,400.0,0.0,REGULAR,25,3.70047,7223927243fc41dfd447834044d1c7dd43fb220effce90...,2.33611,4103,2023-02-01T12:00:15,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",17,2023-02-01T16:30:00,0,15129,137.0
2260,2260,7000,6482,250.0,2023-02-01 10:53:06,ngkumar@berkeley.edu,Model Y,0 days 00:15:52,1139,1,NaN,2023-02-01T10:54:16,400.0,0.0,REGULAR,25,0.86228,87ef316f5ae058c55f21ffa5554f13b820b5b85eb749e5...,0.26444,4104,2023-02-01T11:10:08,NaN,"[{'power_W': Decimal('1636'), 'timestamp': Dec...",13,2023-02-01T17:00:00,0,1575,137.0
1622,1622,100000,6656,250.0,2023-02-01 11:42:52,courtnee.butler@berkeley.edu,Kona,0 days 05:31:45,1149,1,NaN,2023-02-01T11:43:23,400.0,0.0,REGULAR,25,8.07495,2d2ac9072c881728aff5a6b3e67c35880999ed14920dba...,5.52916,4105,2023-02-01T17:15:08,NaN,"[{'power_W': Decimal('3109'), 'timestamp': Dec...",15,2023-02-01T18:00:00,0,34897,137.0


In [21]:
pipeline = Pipeline([
    ("1", SortDropCast()),
    ("2", HelperFeatureCreation()),
    ("3", CreateNestedSessionTimeSeries())
])

In [36]:
raw = pd.read_csv("data/raw_data.csv")
raw = raw.sort_values("connectTime")
raw["connectTime"] = pd.to_datetime(raw["connectTime"])
now = datetime.now().strftime('%D')
filtered = raw[raw["connectTime"] >= now]
clean = pipeline.fit_transform(filtered)
clean = clean.explode(["time_vals", "power_vals"])
clean

,index,Unnamed: 0,vehicle_maxChgRate_W,peakPower_W,sch_centsPerHr,connectTime,vehicle_model,Duration,userId,regular,Deadline,startChargeTime,sch_centsPerOverstayHr,sch_centsPerKwh,choice,siteId,estCost,DurationHrs,dcosId,lastUpdate,energyReq_Wh,power,stationId,defaultDeadline,scheduled,cumEnergy_Wh,reg_centsPerHr,reqChargeTime_h,finishChargeTime,time_vals,power_vals
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:43:09,6893.0
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:48:09,6893.0
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:53:09,6893.0
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:58:09,6893.0
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 08:03:09,6893.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 18:58:36,6278.0
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:03:36,6278.0
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:08:36,6278.0
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:13:36,6278.0


In [38]:
clean["time_vals"] = clean["time_vals"].dt.round('5min')
clean["userId"] = clean["userId"].astype(str)
clean = clean.sort_values("time_vals")
clean

,index,Unnamed: 0,vehicle_maxChgRate_W,peakPower_W,sch_centsPerHr,connectTime,vehicle_model,Duration,userId,regular,Deadline,startChargeTime,sch_centsPerOverstayHr,sch_centsPerKwh,choice,siteId,estCost,DurationHrs,dcosId,lastUpdate,energyReq_Wh,power,stationId,defaultDeadline,scheduled,cumEnergy_Wh,reg_centsPerHr,reqChargeTime_h,finishChargeTime,time_vals,power_vals
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:45:00,6893.0
1,1,2101,7000,3818.0,127.0,2023-02-01 07:43:25,Leaf,0 days 05:16:30,1067,1,NaN,2023-02-01T07:43:40,400.0,0.0,REGULAR,25,10.20599,5.27500,4101,2023-02-01T13:00:10,NaN,"[{'power_W': Decimal('3813'), 'timestamp': Dec...",11,2023-02-01T13:30:00,0,12932.0,184.0,3.387114,2023-02-01 11:06:39,2023-02-01 07:45:00,3818.0
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:50:00,6893.0
1,1,2101,7000,3818.0,127.0,2023-02-01 07:43:25,Leaf,0 days 05:16:30,1067,1,NaN,2023-02-01T07:43:40,400.0,0.0,REGULAR,25,10.20599,5.27500,4101,2023-02-01T13:00:10,NaN,"[{'power_W': Decimal('3813'), 'timestamp': Dec...",11,2023-02-01T13:30:00,0,12932.0,184.0,3.387114,2023-02-01 11:06:39,2023-02-01 07:50:00,3818.0
0,0,552,7200,6893.0,127.0,2023-02-01 07:43:09,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6876'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967.0,184.0,6.668649,2023-02-01 14:23:16,2023-02-01 07:55:00,6893.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:00:00,6278.0
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:05:00,6278.0
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:10:00,6278.0
5,5,80,17200,6278.0,250.0,2023-02-01 15:43:36,Model X,0 days 03:50:04,1180,1,NaN,2023-02-01T15:45:04,400.0,0.0,REGULAR,25,5.75318,3.83444,4106,2023-02-01T19:35:08,NaN,"[{'power_W': Decimal('0'), 'timestamp': Decima...",13,2023-02-01T21:30:00,0,22747.0,137.0,3.623288,2023-02-01 19:21:00,2023-02-01 19:15:00,6278.0


In [39]:
fig = px.bar(clean, x=clean["time_vals"], y=clean["power_vals"], color=clean["userId"])
fig.update_yaxes(showgrid=False)
fig

In [71]:
filtered

,Unnamed: 0,vehicle_maxChgRate_W,peakPower_W,sch_centsPerHr,connectTime,user_email,vehicle_model,Duration,userId,regular,Deadline,startChargeTime,sch_centsPerOverstayHr,sch_centsPerKwh,choice,siteId,estCost,slrpPaymentId,DurationHrs,dcosId,lastUpdate,energyReq_Wh,power,stationId,defaultDeadline,scheduled,cumEnergy_Wh,reg_centsPerHr
6465,6465,7200,6893,127.0,2023-02-01 07:43:09,lee.r@berkeley.edu,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,089cc5605f6f62a8750ca62bf8caca7a62f6a5ac86291f...,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6880'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967,184.0
3486,3486,7200,6893,127.0,2023-02-01 07:43:09,lee.r@berkeley.edu,Bolt,0 days 09:26:50,682,0,2023-02-01T17:30:00,2023-02-01T07:43:19,400.0,0.0,SCHEDULED,25,12.91813,089cc5605f6f62a8750ca62bf8caca7a62f6a5ac86291f...,9.44722,4100,2023-02-01T17:10:09,39580.0,"[{'power_W': Decimal('6880'), 'timestamp': Dec...",16,2023-02-01T17:30:00,1,45967,184.0
8014,8014,7000,3820,127.0,2023-02-01 07:43:25,grosales@berkeley.edu,Leaf,0 days 05:16:30,1067,1,NaN,2023-02-01T07:43:40,400.0,0.0,REGULAR,25,10.20599,277332d81349a2654cad7dbeb0f563b61fecbfaa794fcb...,5.27500,4101,2023-02-01T13:00:10,NaN,"[{'power_W': Decimal('3820'), 'timestamp': Dec...",11,2023-02-01T13:30:00,0,12932,184.0
5035,5035,7000,3820,127.0,2023-02-01 07:43:25,grosales@berkeley.edu,Leaf,0 days 05:16:30,1067,1,NaN,2023-02-01T07:43:40,400.0,0.0,REGULAR,25,10.20599,277332d81349a2654cad7dbeb0f563b61fecbfaa794fcb...,5.27500,4101,2023-02-01T13:00:10,NaN,"[{'power_W': Decimal('3820'), 'timestamp': Dec...",11,2023-02-01T13:30:00,0,12932,184.0
7478,7478,7000,6855,250.0,2023-02-01 09:38:09,stephanie.tomasco@alumni.berkeley.edu,Model Y,0 days 02:20:10,880,1,NaN,2023-02-01T09:40:05,400.0,0.0,REGULAR,25,3.70047,7223927243fc41dfd447834044d1c7dd43fb220effce90...,2.33611,4103,2023-02-01T12:00:15,NaN,"[{'power_W': Decimal('6835'), 'timestamp': Dec...",17,2023-02-01T16:30:00,0,15129,137.0
4499,4499,7000,6855,250.0,2023-02-01 09:38:09,stephanie.tomasco@alumni.berkeley.edu,Model Y,0 days 02:20:10,880,1,NaN,2023-02-01T09:40:05,400.0,0.0,REGULAR,25,3.70047,7223927243fc41dfd447834044d1c7dd43fb220effce90...,2.33611,4103,2023-02-01T12:00:15,NaN,"[{'power_W': Decimal('6835'), 'timestamp': Dec...",17,2023-02-01T16:30:00,0,15129,137.0
8173,8173,7000,6482,250.0,2023-02-01 10:53:06,ngkumar@berkeley.edu,Model Y,0 days 00:15:52,1139,1,NaN,2023-02-01T10:54:16,400.0,0.0,REGULAR,25,0.86228,87ef316f5ae058c55f21ffa5554f13b820b5b85eb749e5...,0.26444,4104,2023-02-01T11:10:08,NaN,"[{'power_W': Decimal('1636'), 'timestamp': Dec...",13,2023-02-01T17:00:00,0,1575,137.0
5194,5194,7000,6482,250.0,2023-02-01 10:53:06,ngkumar@berkeley.edu,Model Y,0 days 00:15:52,1139,1,NaN,2023-02-01T10:54:16,400.0,0.0,REGULAR,25,0.86228,87ef316f5ae058c55f21ffa5554f13b820b5b85eb749e5...,0.26444,4104,2023-02-01T11:10:08,NaN,"[{'power_W': Decimal('1636'), 'timestamp': Dec...",13,2023-02-01T17:00:00,0,1575,137.0
7535,7535,100000,6656,250.0,2023-02-01 11:42:52,courtnee.butler@berkeley.edu,Kona,0 days 05:31:45,1149,1,NaN,2023-02-01T11:43:23,400.0,0.0,REGULAR,25,8.07495,2d2ac9072c881728aff5a6b3e67c35880999ed14920dba...,5.52916,4105,2023-02-01T17:15:08,NaN,"[{'power_W': Decimal('3109'), 'timestamp': Dec...",15,2023-02-01T18:00:00,0,34897,137.0
4556,4556,100000,6656,250.0,2023-02-01 11:42:52,courtnee.butler@berkeley.edu,Kona,0 days 05:31:45,1149,1,NaN,2023-02-01T11:43:23,400.0,0.0,REGULAR,25,8.07495,2d2ac9072c881728aff5a6b3e67c35880999ed14920dba...,5.52916,4105,2023-02-01T17:15:08,NaN,"[{'power_W': Decimal('3109'), 'timestamp': Dec...",15,2023-02-01T18:00:00,0,34897,137.0


In [69]:
car_pie = clean[["dcosId", "vehicle_model"]].groupby("dcosId").first()
car_pie["vehicle_model"].value_counts()

Model Y    2
Bolt       1
Leaf       1
Kona       1
Model X    1
Name: vehicle_model, dtype: int64

In [70]:
px.pie(values = car_pie["vehicle_model"].value_counts(), names=car_pie["vehicle_model"].value_counts().index)